In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.linear_model import Lasso
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error
from scipy.integrate import solve_ivp

import pysindy as ps

# Ignore matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Seed the random number generators for reproducibility
np.random.seed(100)

integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

# Test PDE functionality on 2D Reaction-Diffusion system
This 2D system is significantly more complicated. The reaction-diffusion system exhibits spiral waves on a periodic domain,and the PDEs are:
$$u_t = 0.1\nabla^2 u + (1-A^2)u +\beta A^2v$$
$$v_t = 0.1\nabla^2 v - \beta A^2 u + (1-A^2)v$$
$$A^2 = u^2 + v^2.$$
The main change will be a significantly larger library... cubic terms in (u, v) and all their first and second order derivatives. We will also need to generate the data because saving a high-resolution form of the data makes a fairly large file.

In [ ]:
from numpy.fft import fft2, ifft2
integrator_keywords['method'] = 'RK45'  # switch to RK45 integrator


# Define the reaction-diffusion PDE in the Fourier (kx, ky) space
def reaction_diffusion(t, uvt, K22, d1, d2, beta, n, N):
    ut = np.reshape(uvt[:N], (n, n))
    vt = np.reshape(uvt[N : 2 * N], (n, n))
    u = np.real(ifft2(ut))
    v = np.real(ifft2(vt))
    vu6 = v*u ** 6
    vu4 = v*u ** 4
    vu2 = v*u ** 2
    utrhs = np.reshape((fft2(beta*v + vu2 - vu4 + vu6 - u)), (N, 1)) # Shall we try real term and then get back expansion?
    vtrhs = np.reshape((fft2(-beta*v  - vu2 + vu4 - vu6 + u)), (N, 1))
    uvt_reshaped = np.reshape(uvt, (len(uvt), 1))
    uvt_updated = np.squeeze(
        np.vstack(
            (-d1 * K22 * uvt_reshaped[:N] + utrhs, 
             -d2 * K22 * uvt_reshaped[N:] + vtrhs)
        )
    )
    return uvt_updated


# Generate the data
t = np.linspace(0, 20, int(10 / 0.05))
d1 = 10
d2 = 0.1
beta = 0.067
L = 20  # Domain size in X and Y directions
n = 128  # Number of spatial points in each direction
N = n * n
x_uniform = np.linspace(-L / 2, L / 2, n + 1)
x = x_uniform[:n]
y = x_uniform[:n]
n2 = int(n / 2)
# Define Fourier wavevectors (kx, ky)
kx = (2 * np.pi / L) * np.hstack((np.linspace(0, n2 - 1, n2), 
                                  np.linspace(-n2, -1, n2)))
ky = kx
# Get 2D meshes in (x, y) and (kx, ky)
X, Y = np.meshgrid(x, y)
KX, KY = np.meshgrid(kx, ky)
K2 = KX ** 2 + KY ** 2
K22 = np.reshape(K2, (N, 1))

m = 1  # number of spirals

# define our solution vectors
u = np.zeros((len(x), len(y), len(t)))
v = np.zeros((len(x), len(y), len(t)))

# Initial conditions
u[:, :, 0] = 0.05
v[:, :, 0] = 1-np.tanh(np.sqrt(X ** 2 ))+0.03

In [ ]:
# uvt is the solution vector in Fourier space, so below
# we are initializing the 2D FFT of the initial condition, uvt0
uvt0 = np.squeeze(
    np.hstack(
        (np.reshape(fft2(u[:, :, 0]), (1, N)), 
         np.reshape(fft2(v[:, :, 0]), (1, N)))
    )
)

# Solve the PDE in the Fourier space, where it reduces to system of ODEs
uvsol = solve_ivp(
    reaction_diffusion, (t[0], t[-1]), y0=uvt0, t_eval=t, 
    args=(K22, d1, d2, beta, n, N), **integrator_keywords
)
uvsol = uvsol.y

In [ ]:
# Reshape things and ifft back into (x, y, t) space from (kx, ky, t) space
for j in range(len(t)):
    ut = np.reshape(uvsol[:N, j], (n, n))
    vt = np.reshape(uvsol[N:, j], (n, n))
    u[:, :, j] = np.real(ifft2(ut))
    v[:, :, j] = np.real(ifft2(vt))

In [ ]:
# Plot to check if spiral is nicely reproduced
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.pcolor(X, Y, u[:, :, 2])
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.title('u(x, y, t=0.5)', fontsize=16)
plt.subplot(1, 2, 2)
plt.pcolor(X, Y, v[:, :, 2])
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
ax = plt.gca()
ax.set_yticklabels([])
plt.title('v(x, y, t=0.5)', fontsize=16)

dt = t[1] - t[0]
dx = x[1] - x[0]
dy = y[1] - y[0]

u_sol = u
v_sol = v

In [ ]:
plt.plot([sum(u[50, :, j])+sum(v[50, :, j]) for j in range(200)])
plt.plot([sum(u[50, :, j]) for j in range(200)])
plt.plot([sum(v[50, :, j]) for j in range(200)])
plt.show()

In [ ]:
plt.plot(u[50, :, ::10])
plt.show()
plt.plot(v[50, :, ::10])
plt.show()

In [ ]:
# Compute u_t from generated solution
u = np.zeros((n, n, len(t), 2))
u[:, :, :, 0] = u_sol
u[:, :, :, 1] = v_sol
u_dot = ps.FiniteDifference(axis=2)._differentiate(u, dt)

# Choose 60 % of data for training because data is big... 
# can only randomly subsample if you are passing u_dot to model.fit!!!
train = np.random.choice(len(t), int(len(t) * 0.6), replace=False)
test = [i for i in np.arange(len(t)) if i not in train]
u_train = u[:, :, train, :]
u_test = u[:, :, test, :]
u_dot_train = u_dot[:, :, train, :]
u_dot_test = u_dot[:, :, test, :]
t_train = t[train]
t_test = t[test]
spatial_grid = np.asarray([X, Y]).T

In [ ]:
# Odd polynomial terms in (u, v), up to second order derivatives in (u, v)
library_functions = [
    lambda x: x,
    lambda x, y: y * x * x,
    lambda x, y: y * x * x * x * x,
    lambda x, y: y * x * x * x * x * x * x,
]
library_function_names = [
    lambda x: x,
    lambda x, y: y + x + x,
    lambda x, y: y + x + x + x + x,
    lambda x, y: y + x + x + x + x + x + x,
]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=2,
    spatial_grid=spatial_grid,
    include_bias=True,
    include_interaction=False,
    is_uniform=True,
    periodic=True,
)

In [ ]:
print('STLSQ model: ')
optimizer = ps.STLSQ(threshold=50, alpha=1e-5, 
                     normalize_columns=True, max_iter=200)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(u_train, x_dot=u_dot_train)
model.print()
u_dot_stlsq = model.predict(u_test)

print('SR3 model, L0 norm: ')
optimizer = ps.SR3(
    threshold=60,
    max_iter=1000,
    tol=1e-10,
    nu=1,
    thresholder="l0",
    normalize_columns=True,
)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(u_train, x_dot=u_dot_train)
model.print()
u_dot_sr3 = model.predict(u_test)

print('SR3 model, L1 norm: ')
optimizer = ps.SR3(
    threshold=40,
    max_iter=1000,
    tol=1e-10,
    nu=1e2,
    thresholder="l1",
    normalize_columns=True,
)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(u_train, x_dot=u_dot_train)
model.print()

print('Constrained SR3 model, L0 norm: ')
feature_names = np.asarray(model.get_feature_names())
n_features = len(feature_names)
n_targets = u_train.shape[-1]
constraint_rhs = np.zeros(2)
constraint_lhs = np.zeros((2, n_targets * n_features))

# (u_xx coefficient) - (u_yy coefficient) = 0
constraint_lhs[0, 11] = 1
constraint_lhs[0, 15] = -1
# (v_xx coefficient) - (v_yy coefficient) = 0
constraint_lhs[1, n_features + 11] = 1
constraint_lhs[1, n_features + 15] = -1
optimizer = ps.ConstrainedSR3(
    threshold=.05,
    max_iter=4000,
    tol=1e-10,
    nu=1,
    thresholder="l0",
    normalize_columns=False,
    constraint_rhs=constraint_rhs,
    constraint_lhs=constraint_lhs,
)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(u_train, x_dot=u_dot_train)
model.print()
u_dot_constrained_sr3 = model.predict(u_test)

In [ ]:
feature_names

In [ ]:
u_train.shape[-1]

In [ ]:
n_targets

In [ ]:
n_features

In [ ]:
print('Constrained SR3 model, L0 norm: ')
feature_names = np.asarray(model.get_feature_names())
n_features = len(feature_names)
n_targets = u_train.shape[-1]
constraint_rhs = np.zeros(20)
constraint_lhs = np.zeros((20, n_targets * n_features))

In [ ]:
constraint_lhs[0, 1] = 1
# constraint_lhs[0, n_features + 1] = 1
constraint_rhs[0]=-1

constraint_lhs[1, 2] = 1
# constraint_lhs[1, n_features + 2] = 1
constraint_rhs[1]=0.067

constraint_lhs[2, 3] = 1
constraint_lhs[2, n_features + 3] = 1
constraint_lhs[3, 4] = 1
constraint_lhs[3, n_features + 4] = 1
constraint_lhs[4, 5] = 1
constraint_lhs[4, n_features + 5] = 1
constraint_lhs[5, 6] = 1
constraint_lhs[5, n_features + 6] = 1
constraint_lhs[6, 7] = 1
constraint_lhs[7, n_features + 7] = 1
constraint_lhs[8, n_features + 8] = 1
constraint_lhs[9, 9] = 1
constraint_lhs[10, 11] = 1
constraint_lhs[11, 12] = 1
constraint_lhs[12, 13] = 1
constraint_lhs[13, 14] = 1
constraint_lhs[14, 15] = 1


constraint_lhs[15, 0] = 1
constraint_lhs[16, n_features] = 1
constraint_lhs[17, n_features+1] = 1
constraint_rhs[17] = 1
constraint_lhs[18, n_features+2] = 1
constraint_rhs[18] = -0.067
constraint_lhs[19, 8] = 1
constraint_rhs[19] = 10


In [ ]:
initial_guess = np.array([[0, -1, 0.67, 1, -1, 1, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, -0.67, -1, +1, -1, 0, 0, 0, 0.1, 0, 0, 0, 0, 0, 0]])

optimizer = ps.ConstrainedSR3(
    max_iter=4000,
    tol=1e-10,
    nu=1,
    thresholder="l0",
    normalize_columns=False,
    constraint_rhs=constraint_rhs,
    constraint_lhs=constraint_lhs,
)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(u_train, x_dot=u_dot_train)
model.print()
u_dot_constrained_sr3 = model.predict(u_test)